# common aggreagate function

- summary
- avg,max,min
- sum, sumDistinct
- count,countDistinct
- first, last
- collect_set, collect_list
- skewness
- variance
- stddev

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[*]") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/14 21:32:42 WARN Utils: Your hostname, kirans-mac.local, resolves to a loopback address: 127.0.0.1; using 172.18.197.149 instead (on interface en0)
25/06/14 21:32:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/14 21:32:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Create dataFrame

In [39]:
import datetime
from pyspark.sql import Row

users = [{'id':1,
        'first_name':'kiran',
        'last_name':'chinta',
         'email':'k.c@gmail.com',
         'phone_numbers':Row(mobile=123,home=456),
         'courses':[1,2],
         'is_customer':True,
         'amount_paid':100.55,
         'customer_from':datetime.date(2012,1,1),
        'last_updated_ts':datetime.datetime(2021,2,2,1,2,3)
        },
        {'id':2,
        'first_name':'goats',
        'last_name':'manchi',
         'email':'g.m@gmail.com',
         'phone_numbers':Row(mobile=246,home=680),
         'courses':[11,2],
         'is_customer':False,
         'amount_paid':110.55,
         'customer_from':datetime.date(2012,1,1),
        'last_updated_ts':datetime.datetime(2021,2,2,1,2,3)
        },
        {'id':3,
        'first_name':'ss',
        'last_name':'raj',
         'email':'s.s.r@gmail.com',
         'phone_numbers':Row(mobile=135,home=579),
         'courses':[11,22],
         'is_customer':False,
         'amount_paid':None,
         'customer_from':datetime.date(2012,2,2),
        'last_updated_ts':datetime.datetime(2021,4,5,1,2,3)
        },
        {'id':4,
        'first_name':'puri',
        'last_name':'jag',
         'email':'p.g@gmail.com',
         'phone_numbers':Row(mobile=135,home=579),
         'courses':[11,22],
         'is_customer':False,
         'amount_paid':110.55,
         'customer_from':datetime.date(2012,2,2),
        'last_updated_ts':datetime.datetime(2021,4,5,1,2,3)
        }]

In [40]:
u_df = spark.createDataFrame(users)
u_df.show()

+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+
|amount_paid| courses|customer_from|          email|first_name| id|is_customer|last_name|    last_updated_ts|phone_numbers|
+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+
|     100.55|  [1, 2]|   2012-01-01|  k.c@gmail.com|     kiran|  1|       true|   chinta|2021-02-02 01:02:03|   {123, 456}|
|     110.55| [11, 2]|   2012-01-01|  g.m@gmail.com|     goats|  2|      false|   manchi|2021-02-02 01:02:03|   {246, 680}|
|       NULL|[11, 22]|   2012-02-02|s.s.r@gmail.com|        ss|  3|      false|      raj|2021-04-05 01:02:03|   {135, 579}|
|     110.55|[11, 22]|   2012-02-02|  p.g@gmail.com|      puri|  4|      false|      jag|2021-04-05 01:02:03|   {135, 579}|
+-----------+--------+-------------+---------------+----------+---+-----------+---------+-------------------+-------------+



# summary

In [41]:
u_df.summary().show()

+-------+------------------+---------------+----------+------------------+---------+
|summary|       amount_paid|          email|first_name|                id|last_name|
+-------+------------------+---------------+----------+------------------+---------+
|  count|                 3|              4|         4|                 4|        4|
|   mean|107.21666666666665|           NULL|      NULL|               2.5|     NULL|
| stddev| 5.773502691896258|           NULL|      NULL|1.2909944487358056|     NULL|
|    min|            100.55|  g.m@gmail.com|     goats|                 1|   chinta|
|    25%|            100.55|           NULL|      NULL|                 1|     NULL|
|    50%|            110.55|           NULL|      NULL|                 2|     NULL|
|    75%|            110.55|           NULL|      NULL|                 3|     NULL|
|    max|            110.55|s.s.r@gmail.com|        ss|                 4|      raj|
+-------+------------------+---------------+----------+----------

In [42]:
u_df.select('first_name').summary('count').show()

+-------+----------+
|summary|first_name|
+-------+----------+
|  count|         4|
+-------+----------+



# avg, min , max

In [43]:
from pyspark.sql.functions import avg
u_df.select(avg('amount_paid')).show()

+------------------+
|  avg(amount_paid)|
+------------------+
|107.21666666666665|
+------------------+



In [44]:
from pyspark.sql.functions import min
u_df.select(min('amount_paid')).show()

+----------------+
|min(amount_paid)|
+----------------+
|          100.55|
+----------------+



In [45]:
from pyspark.sql.functions import max
u_df.select(max('amount_paid')).show()

+----------------+
|max(amount_paid)|
+----------------+
|          110.55|
+----------------+



# sum vs sumDistinct

In [46]:
from pyspark.sql.functions import sum, sumDistinct
u_df.select(sum('amount_paid')).show()

+----------------+
|sum(amount_paid)|
+----------------+
|          321.65|
+----------------+



In [47]:
u_df.select(sumDistinct('amount_paid')).show()

+-------------------------+
|sum(DISTINCT amount_paid)|
+-------------------------+
|                    211.1|
+-------------------------+



# count vs countDistinct

In [48]:
from pyspark.sql.functions import count, countDistinct
u_df.select(count('amount_paid')).show()

+------------------+
|count(amount_paid)|
+------------------+
|                 3|
+------------------+



In [50]:
from pyspark.sql.functions import count, countDistinct
u_df.select(countDistinct('amount_paid')).show()

+---------------------------+
|count(DISTINCT amount_paid)|
+---------------------------+
|                          2|
+---------------------------+



# first vs last

In [54]:
from pyspark.sql.functions import first, last
u_df.select(first('amount_paid')).show()

+------------------+
|first(amount_paid)|
+------------------+
|            100.55|
+------------------+



In [55]:
from pyspark.sql.functions import first, last
u_df.select(last('amount_paid')).show()

+-----------------+
|last(amount_paid)|
+-----------------+
|           110.55|
+-----------------+



# collect_set vs collect_list

In [56]:
from pyspark.sql.functions import collect_set, collect_list
u_df.select(collect_set('amount_paid')).show()

+------------------------+
|collect_set(amount_paid)|
+------------------------+
|        [110.55, 100.55]|
+------------------------+



In [58]:
u_df.select(collect_list('amount_paid')).show(truncate=False)

+-------------------------+
|collect_list(amount_paid)|
+-------------------------+
|[100.55, 110.55, 110.55] |
+-------------------------+



# skewness
- Skewness measures the asymmetry of the distribution of values in a column.
    - Positive Skew (Right Skewed): Tail on the right → many small values, few large ones.
	- Negative Skew (Left Skewed): Tail on the left → many large values, few small ones.
	- Skewness ≈ 0: Symmetric distribution (e.g., normal distribution).

In [60]:
from pyspark.sql.functions import skewness

u_df.select(skewness('amount_paid')).show()

+---------------------+
|skewness(amount_paid)|
+---------------------+
|  -0.7071067811865476|
+---------------------+



# variance
	- Variance measures how far the numbers in a dataset are spread out from their mean.
	- High variance → values are more spread out.
	- Low variance → values are closer to the mean.


In [61]:
from pyspark.sql.functions import variance

u_df.select(variance('amount_paid')).show()

+---------------------+
|variance(amount_paid)|
+---------------------+
|   33.333333333333336|
+---------------------+



# stddev
	- Standard deviation is √variance.
	- It measures the average distance of values from the mean.
	- Higher stddev → data more spread out.
	- Lower stddev → data closer to the mean.


In [62]:
from pyspark.sql.functions import stddev

u_df.select(stddev('amount_paid')).show()

+-------------------+
|stddev(amount_paid)|
+-------------------+
|  5.773502691896258|
+-------------------+



In [63]:
spark.stop()